In [1]:
import os
import gdal
import ogr, osr

In [2]:
def UTM47N_to_WebMercator(geom):
    srcSR = osr.SpatialReference()
    srcSR.ImportFromEPSG(32647)          #UTM WGS84 Zone 47N
    destSR = osr.SpatialReference()
    destSR.ImportFromEPSG(3857)          #WGS84 Web Mercator (Auxiliary Sphere)
    coorTrans = osr.CoordinateTransformation(srcSR, destSR)
    new_geom = geom.Clone()
    new_geom.Transform(coorTrans)

    return new_geom

## Step 1: Assgin Boundary (x_min, y_min, x_max, y_max)
### Projection: UTM WGS84 Zone 47N

In [3]:
x_min = 668000
y_min = 1517000
x_max = 668500
y_max = 1517500

#Point 1: x_min, y_min
pt1 = ogr.Geometry(ogr.wkbPoint)
pt1.AddPoint(x_min, y_min)

#Point 2: x_max, y_max
pt2 = ogr.Geometry(ogr.wkbPoint)
pt2.AddPoint(x_max, y_max)

UTM_Bound = (pt1.GetX(), pt1.GetY(), pt2.GetX(), pt2.GetY())

print('Point 1:', pt1.ExportToWkt())
print('Point 2:', pt2.ExportToWkt())
print('Boundary:', UTM_Bound)

Point 1: POINT (668000 1517000 0)
Point 2: POINT (668500 1517500 0)
Boundary: (668000.0, 1517000.0, 668500.0, 1517500.0)


## Step 2: Projection to Web Mercator

In [4]:
#Projection to 
new_pt1 = UTM47N_to_WebMercator(pt1)
new_pt2 = UTM47N_to_WebMercator(pt2)

Web_Bound = (new_pt1.GetX(), new_pt1.GetY(), new_pt2.GetX(), new_pt2.GetY())

print('New Point 1:', new_pt1.ExportToWkt())
print('New Point 2:', new_pt2.ExportToWkt())
print('Boundary:', Web_Bound)

New Point 1: POINT (11193579.7270649 1541801.18348661 0)
New Point 2: POINT (11194097.6451805 1542315.72309877 0)
Boundary: (11193579.727064876, 1541801.1834866055, 11194097.645180535, 1542315.7230987716)


## Step 3: Extract Google Satellite Images with Boundary
### Projection: WGS84 Web Mercator

In [5]:
#Source_WMS = 'Google_Satellite_WMS_Tile.xml'
Source_WMS = 'Google_Maps_WMS_Tile.xml'

Resolution = 0.25

x_min = Web_Bound[0] - 10.0
y_min = Web_Bound[1] - 10.0
x_max = Web_Bound[2] + 10.0
y_max = Web_Bound[3] + 10.0
    
gdal.SetConfigOption('GDAL_HTTP_UNSAFESSL','YES')
OutRS1 = gdal.Warp('GM.tif', Source_WMS, format = 'GTiff',
                   outputBounds = [x_min, y_min, x_max, y_max],
                   xRes = Resolution, yRes = Resolution)


## Step 4: Raster Projection from Web Mercator to UTM WGS84 Zone47

In [6]:
#Outfn = 'Google_Satellite.tif' #Should be Mapsheet No.
Outfn = 'Google_Maps.tif' #Should be Mapsheet No.

OutRS2 = gdal.Warp(Outfn, OutRS1, format='GTiff', dstSRS='EPSG:32647')

## Step 5: Clip Boundary

In [7]:
Resolution = 0.25 #Case Google Maps Change Resolution = xxxx
#Final_fn = 'Google_Satellite_UTM.tif'
Final_fn = 'Google_Maps_UTM.tif'

x_min = UTM_Bound[0]
y_min = UTM_Bound[1]
x_max = UTM_Bound[2]
y_max = UTM_Bound[3]
    
gdal.SetConfigOption('GDAL_HTTP_UNSAFESSL','YES')
OutRS3 = gdal.Warp(Final_fn, OutRS2, format='GTiff',
                   outputBounds = [x_min, y_min, x_max, y_max],
                   xRes = Resolution, yRes = Resolution)

In [8]:
OutRS1 = None
OutRS2 = None
OutRS3 = None

#Delete temporary File
if os.path.exists('GM.tif'):
    os.remove('GM.tif')
    
if os.path.exists('Google_Maps.tif'):
    os.remove('Google_Maps.tif')

if os.path.exists('Google_Satellite.tif'):
    os.remove('Google_Satellite.tif')